# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 19 2022 8:05AM,237607,19,ADV80002751,"AdvaGen Pharma, Ltd",Released
1,May 19 2022 8:05AM,237607,19,ADV80002752,"AdvaGen Pharma, Ltd",Released
2,May 19 2022 8:05AM,237607,19,ADV80002753,"AdvaGen Pharma, Ltd",Released
3,May 19 2022 8:05AM,237607,19,ADV80002754,"AdvaGen Pharma, Ltd",Released
4,May 19 2022 8:05AM,237607,19,ADV80002755,"AdvaGen Pharma, Ltd",Released
5,May 19 2022 8:05AM,237607,19,ADV80002756,"AdvaGen Pharma, Ltd",Released
6,May 19 2022 8:05AM,237607,19,ADV80002759,"AdvaGen Pharma, Ltd",Released
7,May 19 2022 8:05AM,237607,19,ADV80002760,"AdvaGen Pharma, Ltd",Released
8,May 19 2022 8:05AM,237607,19,ADV80002761,"AdvaGen Pharma, Ltd",Released
9,May 19 2022 8:05AM,237607,19,ADV80002762,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
44,237601,Released,1
45,237602,Released,1
46,237604,Released,1
47,237606,Executing,1
48,237607,Released,43


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
237601,NaN,NaN,NaN,1.0
237602,NaN,NaN,NaN,1.0
237604,NaN,NaN,NaN,1.0
237606,NaN,NaN,1.0,NaN
237607,NaN,NaN,NaN,43.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
237507,1.0,13.0,2.0,0.0
237510,0.0,3.0,0.0,1.0
237513,0.0,0.0,0.0,1.0
237514,0.0,0.0,0.0,1.0
237529,0.0,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
237507,1.0,13,2,0
237510,0.0,3,0,1
237513,0.0,0,0,1
237514,0.0,0,0,1
237529,0.0,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,237507,1.0,13,2,0
1,237510,0.0,3,0,1
2,237513,0.0,0,0,1
3,237514,0.0,0,0,1
4,237529,0.0,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,237507,1.0,13,2,
1,237510,0.0,3,,1
2,237513,0.0,,,1
3,237514,0.0,,,1
4,237529,0.0,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 19 2022 8:05AM,237607,19,"AdvaGen Pharma, Ltd"
43,May 19 2022 7:54AM,237606,16,"SHL Pharma, LLC"
44,May 19 2022 7:47AM,237604,16,"SD Head USA, LLC"
45,May 18 2022 5:03PM,237602,16,American International Chemical
46,May 18 2022 4:54PM,237601,16,American International Chemical
47,May 18 2022 4:39PM,237596,18,"Blinc International, Inc. - PV"
48,May 18 2022 4:38PM,237600,10,ISDIN Corporation
49,May 18 2022 4:07PM,237593,10,"Methapharm, Inc."
53,May 18 2022 4:02PM,237590,18,"Blinc International, Inc."
54,May 18 2022 3:54PM,237587,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,May 19 2022 8:05AM,237607,19,"AdvaGen Pharma, Ltd",0.0,,,43
1,May 19 2022 7:54AM,237606,16,"SHL Pharma, LLC",0.0,,1,
2,May 19 2022 7:47AM,237604,16,"SD Head USA, LLC",0.0,,,1
3,May 18 2022 5:03PM,237602,16,American International Chemical,0.0,,,1
4,May 18 2022 4:54PM,237601,16,American International Chemical,0.0,,,1
5,May 18 2022 4:39PM,237596,18,"Blinc International, Inc. - PV",0.0,,,1
6,May 18 2022 4:38PM,237600,10,ISDIN Corporation,0.0,,,1
7,May 18 2022 4:07PM,237593,10,"Methapharm, Inc.",0.0,1,1,2
8,May 18 2022 4:02PM,237590,18,"Blinc International, Inc.",0.0,,,1
9,May 18 2022 3:54PM,237587,20,Alumier Labs Inc.,0.0,,2,33


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 8:05AM,237607,19,"AdvaGen Pharma, Ltd",43,,
1,May 19 2022 7:54AM,237606,16,"SHL Pharma, LLC",,1,
2,May 19 2022 7:47AM,237604,16,"SD Head USA, LLC",1,,
3,May 18 2022 5:03PM,237602,16,American International Chemical,1,,
4,May 18 2022 4:54PM,237601,16,American International Chemical,1,,
5,May 18 2022 4:39PM,237596,18,"Blinc International, Inc. - PV",1,,
6,May 18 2022 4:38PM,237600,10,ISDIN Corporation,1,,
7,May 18 2022 4:07PM,237593,10,"Methapharm, Inc.",2,1,1
8,May 18 2022 4:02PM,237590,18,"Blinc International, Inc.",1,,
9,May 18 2022 3:54PM,237587,20,Alumier Labs Inc.,33,2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 8:05AM,237607,19,"AdvaGen Pharma, Ltd",43,,
1,May 19 2022 7:54AM,237606,16,"SHL Pharma, LLC",,1,
2,May 19 2022 7:47AM,237604,16,"SD Head USA, LLC",1,,
3,May 18 2022 5:03PM,237602,16,American International Chemical,1,,
4,May 18 2022 4:54PM,237601,16,American International Chemical,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 8:05AM,237607,19,"AdvaGen Pharma, Ltd",43.0,NaN,NaN
1,May 19 2022 7:54AM,237606,16,"SHL Pharma, LLC",NaN,1.0,NaN
2,May 19 2022 7:47AM,237604,16,"SD Head USA, LLC",1.0,NaN,NaN
3,May 18 2022 5:03PM,237602,16,American International Chemical,1.0,NaN,NaN
4,May 18 2022 4:54PM,237601,16,American International Chemical,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 8:05AM,237607,19,"AdvaGen Pharma, Ltd",43.0,0.0,0.0
1,May 19 2022 7:54AM,237606,16,"SHL Pharma, LLC",0.0,1.0,0.0
2,May 19 2022 7:47AM,237604,16,"SD Head USA, LLC",1.0,0.0,0.0
3,May 18 2022 5:03PM,237602,16,American International Chemical,1.0,0.0,0.0
4,May 18 2022 4:54PM,237601,16,American International Chemical,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2375626,57.0,10.0,20.0
12,237541,0.0,1.0,0.0
16,1663064,10.0,16.0,0.0
18,1187826,7.0,2.0,13.0
19,712750,45.0,0.0,0.0
20,1662890,110.0,3.0,0.0
21,475151,2.0,0.0,0.0
22,237571,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2375626,57.0,10.0,20.0
1,12,237541,0.0,1.0,0.0
2,16,1663064,10.0,16.0,0.0
3,18,1187826,7.0,2.0,13.0
4,19,712750,45.0,0.0,0.0
5,20,1662890,110.0,3.0,0.0
6,21,475151,2.0,0.0,0.0
7,22,237571,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,57.0,10.0,20.0
1,12,0.0,1.0,0.0
2,16,10.0,16.0,0.0
3,18,7.0,2.0,13.0
4,19,45.0,0.0,0.0
5,20,110.0,3.0,0.0
6,21,2.0,0.0,0.0
7,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,57.0
1,12,Released,0.0
2,16,Released,10.0
3,18,Released,7.0
4,19,Released,45.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20,21,22
Status,,,,,,,,
Completed,20.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0
Executing,10.0,1.0,16.0,2.0,0.0,3.0,0.0,1.0
Released,57.0,0.0,10.0,7.0,45.0,110.0,2.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20,21,22
0,Completed,20.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0
1,Executing,10.0,1.0,16.0,2.0,0.0,3.0,0.0,1.0
2,Released,57.0,0.0,10.0,7.0,45.0,110.0,2.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20,21,22
0,Completed,20.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0
1,Executing,10.0,1.0,16.0,2.0,0.0,3.0,0.0,1.0
2,Released,57.0,0.0,10.0,7.0,45.0,110.0,2.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()